In [2]:
#Create a model for Multiple Linear Regression
import pandas as pd

df = pd.read_csv('insurance_pre.csv')
df = pd.get_dummies(df, drop_first=True)
#df = pd.get_dummies(df)
df.head()

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False


In [3]:
df.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [4]:
# Split Input and Output
x_values = df.iloc[:,df.columns != 'charges']
y_values = df.iloc[:,df.columns == 'charges']

In [5]:
# split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.2)

In [6]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150,200],
    'learning_rate': [0.01, 0.1, 1.0],
    'loss': ['linear', 'square', 'exponential']
}

In [ ]:
#create a Decision Tree model
from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn.ensemble import AdaBoostRegressor as abr
from sklearn.model_selection import GridSearchCV as gsc

# Base estimator 
base_estimator = dtr()
abr_model = abr(estimator=base_estimator, random_state=42)

# Use GridSearchCV to search best hyperparameters
grid_search = gsc(
    estimator=abr_model,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

# Fit the model on the training data
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [ ]:
# Best parameters
print("Best Parameters Found:", grid_search.best_params_)

Best Parameters Found: {'learning_rate': 0.01, 'loss': 'linear', 'n_estimators': 50}


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import r2_score

# Predict the output for the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
# Evaluate on test data
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Test MSE:", mse)
print("Test RMSE:", np.sqrt(mse))
print("R-squared:", r2)

Test MSE: 30592095.00743872
Test RMSE: 5531.012114201046
R-squared: 0.7917656220609814


In [ ]:
#save the model using pickle
import pickle as pkl
with open('abr_model.sav', 'wb') as file:
    pkl.dump(grid_search, file)

# Load the model
with open('abr_model.sav', 'rb') as file:
    loaded_model = pkl.load(file)
# Predict using the loaded model
loaded_y_pred = loaded_model.predict([[46,23.8,2,1,0]])
print("Prediction using loaded model:", loaded_y_pred)

NameError: name 'grid_search' is not defined